In [1]:
import pandas as pd
train_file = "train.data.txt"
dev_file = "dev.data.txt"
test_file = "test.data.txt"


train_data = pd.read_csv('./%s.csv'%train_file,keep_default_na=False)
dev_data = pd.read_csv('./%s.csv'%dev_file,keep_default_na=False)
test_data = pd.read_csv('./%s.csv'%test_file,keep_default_na=False)


In [2]:
from transformers import BertTokenizer
import numpy as np
import json
from sklearn.preprocessing import LabelEncoder
import re
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
le = LabelEncoder()

def preprocess_token(df):
    data = df.copy()
    data['main_tweet'] = data['main_tweet'].fillna('')
    data.replace('', np.nan, inplace=True)
    data.dropna(subset=['main_tweet'], inplace=True)
    data['main_tweet'] = data['main_tweet'].apply(lambda x: re.sub('http\S+', '', x))
    data['main_tweet'] = data['main_tweet'].apply(lambda x: x.replace('\n', ' ')) 
    texts = []
    for main_tweet, replies in zip(data['main_tweet'], data['replies']):
        text = main_tweet + ' [SEP] '
        replies = json.loads(replies)
        for reply in replies:
            text += reply['tweet'] + ' [SEP] '
        texts.append(text)
    
    delta = pd.DataFrame({'text': texts, 'label': data['label']}, index=data.index)
    return delta
train = preprocess_token(train_data)
train['label']=le.fit_transform(train['label'])
encoded = tokenizer(train['text'].tolist(), add_special_tokens=True, truncation=True,padding=True, return_tensors="tf")
# tokenizer.convert_ids_to_tokens(encoding['input_ids'])
vocan_length = len(tokenizer.vocab)
train['text']
encoded['input_ids']

Metal device set to: AMD Radeon Pro 555

systemMemory: 16.00 GB
maxCacheSize: 1.00 GB



2022-04-27 20:31:09.215231: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-27 20:31:09.218764: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-27 20:31:09.219511: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<tf.Tensor: shape=(1566, 512), dtype=int32, numpy=
array([[   101,    126,    119, ...,  37715,  10251,    102],
       [   101,  11894,  15034, ...,  10657, 103896,    102],
       [   101,  32769,  37715, ...,      0,      0,      0],
       ...,
       [   101,    107,  57246, ...,      0,      0,      0],
       [   101,  18051,  10529, ...,      0,      0,      0],
       [   101,    137,  66003, ...,      0,      0,      0]], dtype=int32)>

In [3]:
from transformers import TFBertModel
model = TFBertModel.from_pretrained('bert-base-multilingual-cased', num_labels=2)
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(encoded['input_ids'], train['label'], epochs=3)


Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  177853440 
                                                                 
Total params: 177,853,440
Trainable params: 177,853,440
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3


2022-04-27 20:31:39.992594: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


InvalidArgumentError: Graph execution error:

Detected at node 'binary_crossentropy/mul' defined at (most recent call last):
    File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/kan/tensorflow-metal/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/kan/tensorflow-metal/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Users/kan/tensorflow-metal/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/kan/tensorflow-metal/lib/python3.8/site-packages/tornad